# Рекомендация тарифов

# Описание проекта

Оператор мобильной связи выяснил: многие клиенты пользуются архивными тарифами. 
Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. 
Нужно построить модель для задачи классификации, которая выберет подходящий тариф. 

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Откроем-и-изучим-файл" data-toc-modified-id="Откроем-и-изучим-файл-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Откроем и изучим файл</a></span></li><li><span><a href="#Разобьём-данные-на-выборки" data-toc-modified-id="Разобьём-данные-на-выборки-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Разобьём данные на выборки</a></span></li><li><span><a href="#Исследуем-модели" data-toc-modified-id="Исследуем-модели-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Исследуем модели</a></span><ul class="toc-item"><li><span><a href="#Дерево-принятия-решений" data-toc-modified-id="Дерево-принятия-решений-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Дерево принятия решений</a></span></li><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Случайный лес</a></span></li><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li></ul></li><li><span><a href="#Проверим-модель-на-тестовой-выборке" data-toc-modified-id="Проверим-модель-на-тестовой-выборке-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверим модель на тестовой выборке</a></span></li><li><span><a href="#Проверим-модели-на-адекватность" data-toc-modified-id="Проверим-модели-на-адекватность-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Проверим модели на адекватность</a></span></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Общий вывод</a></span></li></ul></div>

## Откроем и изучим файл

In [1]:
# Импорт библиотек
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier

In [2]:
df = pd.read_csv('datasets/users_behavior.csv') #открываем файл

In [3]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


###### Вывод:

В наших руках обработанные данные без пропусков и готовые к обучению.

## Разобьём данные на выборки

In [5]:
#разабьем данные на признаки и целевой признак
features = df.drop(['is_ultra'], axis=1) #признаки
target = df['is_ultra'] #целевой признак

In [6]:
# Разделим исходные данные на обучающую, валидационную и тестовую выборки 
features_df, features_valid, target_df, target_valid = train_test_split(
    features, target, test_size=0.20, random_state=12345, stratify = target)

features_train, features_test, target_train, target_test = train_test_split(
    features_df, target_df, test_size=0.25, random_state=12345, stratify = target_df)

In [7]:
print(features_train.shape)
print(target_train.shape)
print(features_test.shape)
print(target_test.shape)
print(features_valid.shape)
print(target_valid.shape)

(1928, 4)
(1928,)
(643, 4)
(643,)
(643, 4)
(643,)


## Исследуем модели

### Дерево принятия решений 

###### Попробуем получить лучшую глубину дерева с помощью цикла

In [8]:
best_model = None
best_result = 0
for depth in range(1, 80):
    model_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth) 
    model_tree.fit(features_train, target_train) 
    predictions = model_tree.predict(features_valid)
    result = accuracy_score(target_valid, predictions)
    if result > best_result:
        best_model = model_tree
        best_result = result
        best_depth= max_depth = depth
        
print('Accuracy наилучшей модели на валидационной выборке:', best_result)
print('Лучшая глубина дерева:', best_depth)        

Accuracy наилучшей модели на валидационной выборке: 0.7993779160186625
Лучшая глубина дерева: 3


In [9]:
best_tree_model = DecisionTreeClassifier(random_state=12345, max_depth=3) # лучшая модель из дерева принятия решений
best_tree_model.fit(features_train, target_train);

### Случайный лес 

###### Выберем гиперпараметры с помощью цикла в случайном лесу

In [10]:
best_model = None
best_result = 0
best_est = 0
best_depth = 0
for est in range(10, 100, 10):
    for depth in range(1, 15):
        model_forest = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth) 
        model_forest.fit(features_train, target_train)
        predictions = model_forest.predict(features_valid)
        result = accuracy_score(target_valid, predictions) 
        if result > best_result:
                best_model = model_forest
                best_result = result
                best_est = est
                best_depth = depth
print('Accuracy наилучшей модели на валидационной выборке:', best_result)
print('Лучшее дерево', best_est)
print('Лучшее глубина дерева', best_depth)

Accuracy наилучшей модели на валидационной выборке: 0.8242612752721618
Лучшее дерево 50
Лучшее глубина дерева 11


In [11]:
best_forest_model = RandomForestClassifier(random_state=12345, n_estimators=50, max_depth=11) # лучшая модель из рандомного леса
best_forest_model.fit(features_train, target_train);

### Логистическая регрессия  

###### Обучим модель с помощью логической регрессии

In [12]:
model_logistic = LogisticRegression(random_state=12345, solver='lbfgs')
model_logistic.fit(features_train, target_train) 

result_train = model_logistic.score(features_train, target_train)
result_valid = model_logistic.score(features_valid, target_valid)

print('Accuracy модели логистической регрессии на обучающей выборке:', result_train)
print('Accuracy модели логистической регрессии на валидационной выборке:', result_valid)

Accuracy модели логистической регрессии на обучающей выборке: 0.7463692946058091
Accuracy модели логистической регрессии на валидационной выборке: 0.7371695178849145


###### Значения аccuracy на обучающей и  валидационной выборки слишкмо малы. Такая модель нам не подходит, но  у логической регресии есть свой плюс, никакого переобучения.

## Проверим модель на тестовой выборке

###### Для проверки моделей мы выбрали accuracy.
    Каждое неправильное предсказание — это неверная подсказка и потенциальная упущенная выгода для продавца.
    И наоборот: чем выше точность классификации, тем больше прибыли принесёт продукт.

In [13]:
accuracy_tree = best_tree_model.score(features_test, target_test)
print('Accuracy модели дерева принятия решений на тестовой выборке:' ,accuracy_tree)

accuracy_forest =  best_forest_model.score(features_test, target_test)
print('Accuracy модели случайного леса на тестовой выборке:' ,accuracy_forest)

Accuracy модели дерева принятия решений на тестовой выборке: 0.8040435458786936
Accuracy модели случайного леса на тестовой выборке: 0.8195956454121306


###### Метрика качества аccuracy показывает, что обе модели показали хорошие результаты на тестовой выборке.
    Но лучшей стала модель случайного леса.

## Проверим модели на адекватность

Для проверки модели на адекватность используем `DummyClassifier`

`DummyClassifier`- это классификатор, который делает прогнозы с использованием простых правил.

Этот классификатор полезен в качестве простого сравнения с другими классификаторами.

Если наши алгоритмы лучше чем рандомный ответ, то модель прошла проверку на адекватность.

In [14]:
dummy = DummyClassifier(strategy='most_frequent', random_state=12345)
dummy.fit(features_train, target_train)

dummy_train =  dummy.score(features_train, target_train)
dummy_valid =  dummy.score(features_valid, target_valid)
dummy_test =  dummy.score(features_test, target_test)

print('Сравнение обучабщей выборки :', dummy_train)
print('Сравнение валидационной выборки :',dummy_valid)
print('Сравнение тестовой выборки :',dummy_test)

Сравнение обучабщей выборки : 0.6934647302904564
Сравнение валидационной выборки : 0.6936236391912908
Сравнение тестовой выборки : 0.6936236391912908


In [15]:
train_forest =  best_forest_model.score(features_train, target_train)
valid_forest =  best_forest_model.score(features_valid, target_valid)
test_forest =  best_forest_model.score(features_test, target_test)

print('Значение лучшей модели на обучающей выборке :', train_forest)
print('Значение лучшей модели на валидационной выборке :', valid_forest)
print('Значение лучшей модели на тестовой выборке :', test_forest)

Значение лучшей модели на обучающей выборке : 0.8983402489626556
Значение лучшей модели на валидационной выборке : 0.8242612752721618
Значение лучшей модели на тестовой выборке : 0.8195956454121306


## Общий вывод

Мы построили модель для задачи классификации, которая выберет подходящий тариф.

Лучшей моделью стал случайный лес с параметрами `n_estimators=50`, `max_depth=11`. 

На тестовой выборке мы получили значение `accuracy = 0.8195956454121306`

При проверке модели на адекватность, получившиеся значения были больше случайно сгенерированных с помощью `DummyClassifier`, тем самым доказывая адекватность нашей модели.